In [0]:
"""
Changes done on May 17th by Sharath
a)Loading images - written function to load the images, used tqdm to show the progress, it also allows us to resize the image size required
b)Display output image - written code to merge the actual and predicted image, so it can be shown side by side.  this is written as cv.imshow 
    won't work in collab, instead we need to use cv2_imshow which doesnot allow printing side by side images
c)Model History graphs - Not yet tested - added code to show the model history on accuracy and loss
changes done on 24th May by sharath
d)Excel Reporting - added excel reporting to log the run details on each run
Changes done on 24th May by Pratosh
e)data_convert - function changed to run faster
f)Parameter auto run - added excel to track the assignment of runs, so code picks up next combination to run - sharath 24th May
g)added earlystopping on epochs
h)added IoU metrics based on Pratosh input to model.compile
i)added exit at the end to restart the colab each time running
"""

"\nChanges done on May 17th by Sharath\na)Loading images - written function to load the images, used tqdm to show the progress, it also allows us to resize the image size required\nb)Display output image - written code to merge the actual and predicted image, so it can be shown side by side.  this is written as cv.imshow \n    won't work in collab, instead we need to use cv2_imshow which doesnot allow printing side by side images\nc)Model History graphs - Not yet tested - added code to show the model history on accuracy and loss\nchanges done on 24th May by sharath\nd)Excel Reporting - added excel reporting to log the run details on each run\nChanges done on 24th May by Pratosh\ne)data_convert - function changed to run faster\nf)Parameter auto run - added excel to track the assignment of runs, so code picks up next combination to run - sharath 24th May\ng)added earlystopping on epochs\nh)added IoU metrics based on Pratosh input to model.compile\ni)added exit at the end to restart the c

In [0]:
#set the path of images and log files
path_y = "/content/drive/My Drive/CapstoneImg/seg/seg_f/"
path_x = "/content/drive/My Drive/CapstoneImg/rgb/rgb_f/"
logPath="/content/drive/My Drive/CapstoneImg/"
runnerName="Sharath"
n_classes = 12

In [1]:
from keras.models import Model
from keras.layers import Add,Input,concatenate,BatchNormalization
from keras.regularizers import l2
from keras.applications import *
from keras.layers.convolutional import Conv2D,Conv2DTranspose,MaxPooling2D
from keras.layers.core import Activation
import tensorflow as tf
from keras.backend import tensorflow_backend as K
from keras.utils.layer_utils import count_params
from keras.callbacks import EarlyStopping
import numpy as np
import cv2
import glob
import os
from tqdm import tqdm #for progress bars while running to check the current progess of execution
#time operation
from datetime import datetime
import time
#Excel operations
import openpyxl
from openpyxl import Workbook
from os import path

from google.colab import drive
drive.mount('/content/drive') 

Using TensorFlow backend.


In [0]:
# datetime object containing current date and time
stTime = datetime.now()
# dd/mm/YY H:M:S
dt_string = stTime.strftime("%d/%m/%Y %H:%M:%S")

In [0]:
# Read all parameters - remembers last run and hence it picks the next combination
lngFound=0 #setting 0 to indicate the model not assigned
lngResetRow=0 #Set this to excel row value if we want a combination parameter to be rerun (overrides the sequence)
if (path.exists(logPath+"ModelParameters.xlsx")==True): #File exist, open it
  wbP = openpyxl.load_workbook(logPath+"ModelParameters.xlsx")
  wsP = wbP.active
  colummn=wsP['A']
  lastRow=len(colummn)+1
  lastRun=wsP['B1'].value
  for i in range(lastRow):#loop through all rows to identify next run
    if (i==0): i=1
    if (int(lastRun)<i or lngResetRow==i): #Found a row for run
      whoAmI=wsP['H'+str(i)].value
      if (whoAmI==runnerName or lngResetRow==i):       
        #read all run parameters and exit. else continue searching for the row
        strModelName=wsP['A'+str(i)].value
        strBaseModel=wsP['B'+str(i)].value
        lngepochs=int(wsP['C'+str(i)].value)
        lngTrainsize=int(wsP['D'+str(i)].value)
        lngValSize=int(wsP['E'+str(i)].value)
        lngTestSize=int(wsP['F'+str(i)].value)
        lngImageSize=(wsP['G'+str(i)].value)
        strBatchNorm=wsP['I'+str(i)].value
        lngFrozenLayers=int(wsP['J'+str(i)].value)
        strLossFunction=wsP['K'+str(i)].value
        wsP['B1']=i
        lngFound=i #setting to 1 to indicate model is set
        break
if (lngFound==0):#Not found, set with default values
  strModelName="FCN8"
  strBaseModel="vgg16"
  strBaseModel="vgg16"
  lngepochs=50
  lngTrainsize=300
  lngValSize=50
  lngTestSize=50
  lngImageSize=1
  strBatchNorm="No"
  lngFrozenLayers=5
  strLossFunction="focal"

In [0]:
  lngFound,strModelName,  strBaseModel,  lngepochs,  lngTrainsize,  lngValSize,  lngTestSize,lngImageSize

(5, 'Unet', None, 50, 300, 50, 20, 0.75)

In [0]:
class UNET:

   class unet:
        """ Unet architecture

               Usage :
                   unet = Semantic_Segmentation.models.UNET.unet(input_shape,no_classes,regularizer,summary)
                   model = unet.build()

               # Arguments
                   input_shape : size of the input image ,in tuple.
                   n_classes : the number of target class.
                               dtype --> int
                   regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                               dtype --> float   default -->None
                   summary : If True prints the model summary
                                default --> True
           """
        build = lambda self: self.__architecture()
        weight_decay = lambda self, x: None if x == None else l2(x)
        batchnorm = lambda self,x: BatchNormalization(beta_regularizer=l2(0.001),gamma_regularizer=l2(0.001)) if x else Activation('linear')

        def __init__(self,input_shape,n_classes,regularizer=None,BatchNorm=True,summary=True):
            self.input_shape = input_shape
            self.n_classes = n_classes
            self.regularizer = regularizer
            self.BatchNorm = BatchNorm
            self.summary = summary


        def __architecture(self):

            input = Input(self.input_shape)

            conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(input)
            conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv1)
            batchnorm1 = self.batchnorm(self.BatchNorm)(conv1)
            pool1 = MaxPooling2D(pool_size=(2, 2))(batchnorm1)
             
            
            conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(pool1)
            conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv2)
            batchnorm2 = self.batchnorm(self.BatchNorm)(conv2)
            pool2 = MaxPooling2D(pool_size=(2, 2))(batchnorm2)

            
            conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(pool2)
            conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv3)
            batchnorm3 = self.batchnorm(self.BatchNorm)(conv3)
            pool3 = MaxPooling2D(pool_size=(2, 2))(batchnorm3)

            
            conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(pool3)
            conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv4)
            batchnorm4 = self.batchnorm(self.BatchNorm)(conv4)
            pool4 = MaxPooling2D(pool_size=(2, 2))(batchnorm4)

            conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(pool4)
            conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv5)
            batchnorm5 = self.batchnorm(self.BatchNorm)(conv5)
            up5 = Conv2DTranspose(512, 4, strides=(2,2), padding='same', kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_5')(batchnorm5)

            merge6 = concatenate([batchnorm4, up5], axis=3)
            conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=self.weight_decay(self.regularizer))(merge6)
            conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=self.weight_decay(self.regularizer))(conv6)
            batchnorm6 = self.batchnorm(self.BatchNorm)(conv6)
            up6 = Conv2DTranspose(256, 4, strides=(2,2), padding='same', kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_6')(batchnorm6)

            merge7 = concatenate([batchnorm3, up6], axis=3)
            conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=self.weight_decay(self.regularizer))(merge7)
            conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv7)
            batchnorm7 = self.batchnorm(self.BatchNorm)(conv7)
            up7 = Conv2DTranspose(128, 4, strides=(2,2), padding='same', kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_7')(batchnorm7)

            merge8 = concatenate([batchnorm2, up7], axis=3)
            conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
            conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
            batchnorm8 = self.batchnorm(self.BatchNorm)(conv8)
            up8 = Conv2DTranspose(64, 4, strides=(2,2), padding='same', kernel_regularizer=self.weight_decay(self.regularizer), name='upsample_8')(batchnorm8)

            merge9 = concatenate([batchnorm1, up8], axis=3)
            conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(merge9)
            conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv9)
            conv9 = Conv2D(self.n_classes, 1, activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=self.weight_decay(self.regularizer))(conv9)
            op = Activation('softmax',name='softmax')(conv9)

            model = Model(input=input, output=op,name='Unet')
            if self.summary: print(model.summary())
            return model






class FCN:
    """ Available models --> fcn32, fcn16, fcn8

        All the three avilable with vgg16,vgg19,resnet50
    """

    class fcn32:

        """ FCN32 architecture, it doesn't use any skip connections and upsamples the image by the scale of 32.

            Usage :
                FCN32 = Semantic_Segmentation.models.FCN.fcn32(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN32.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """
        weight_decay = lambda self,x: l2(x) if type(x)==int else None
        build = lambda self: self.__decoder(self.encoder())

        def __init__(self,input_shape,n_classes,base_model='vgg16',weight_path='imagenet',regularizer = None,summary=True):
            self.base_model = base_model
            self.input_shape = input_shape
            self.n_classes = n_classes
            self.weightpath = weight_path
            self.regularizer = regularizer
            self.summary = summary


        def encoder(self):

            """ Builds the encoder layer on top of the given pre-trained CNN model

                # Returns
                    model_en : the architecture of the encoder part
            """
            if self.base_model == 'vgg16':
                base = vgg16.VGG16(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)
            elif self.base_model == 'vgg19':
                base = vgg19.VGG19(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)
            else: base = resnet50.ResNet50(include_top=False, weights=self.weightpath, pooling=None, input_shape=self.input_shape)

            layer_encoder = Conv2D(4096, (7, 7), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name='conv_en1')(base.output)
            layer_encoder = Conv2D(4096, (1, 1), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name='conv_en2')(layer_encoder)

            encoder_model = Model(base.input,layer_encoder)
            return encoder_model


        def __decoder(self,model_en):

            """ Builds the decoder layer on top of the encoder part.

                # Arguments
                    model_en : the model architecture of the encoder part

                # Returns
                    model : the full model architecture, both encoder and decoder combined
            """
            layer_decoder = Conv2D(self.n_classes, (1,1), padding='same', activation='relu', kernel_regularizer=self.weight_decay(self.regularizer),
                                   name = 'conv_dec')(model_en.output)
            layer_decoder = Conv2DTranspose(self.n_classes, (64,64), strides=(32,32), padding = 'same', kernel_regularizer = self.weight_decay(self.regularizer),
                                            name = 'deconv')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input,output,name='FCN32-'+self.base_model)
            if self.summary: print(model.summary())
            return model


        def skip_connections(self,model_en):

            """ Builds the skip connections based on the pre-trained CNN models for FCN16 and FCN8.
                NOTE- FCN32 doesnt use this function, its a base for other class/FCN16,FCN8

                # Arguments:
                    model_en : the model architecture of the encoder part
            """
            if self.base_model == 'resnet50': skip_1,skip_2 = model_en.get_layer(model_en.layers[112].name).output, model_en.get_layer(model_en.layers[50].name).output
            elif self.base_model == 'vgg16': skip_1,skip_2 = model_en.get_layer(model_en.layers[14].name).output, model_en.get_layer(model_en.layers[10].name).output
            else: skip_1,skip_2 = model_en.get_layer(model_en.layers[16].name).output, model_en.get_layer(model_en.layers[11].name).output
            return skip_1,skip_2






    class fcn16(fcn32):

        """ FCN16 architecture, it uses one skip connection and upsamples the image by the scale of 16.

            Usage :
                FCN16 = Semantic_Segmentation.models.FCN.fcn16(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN16.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """

        ## NOTE : It wraps the properties of FCN32, same arguments and encoder part but it uses diff decoder function
        ##since the decoder includes skip connection.
        build = lambda self: self.__decoder(super().encoder())

        def __decoder(self,model_en):

            skip,_ = super().skip_connections(model_en)

            layer_decoder = Conv2DTranspose(skip.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv1')(model_en.output)
            layer_decoder = Add(name='skip1')([layer_decoder, skip])
            layer_decoder = Conv2DTranspose(self.n_classes, (32,32), strides=(16, 16), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv3')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input, output,name='FCN16-'+self.base_model)
            if self.summary: print(model.summary())
            return model




    class fcn8(fcn32):

        """ FCN8 architecture, it uses two skip connections and upsamples the image by the scale of 8, preserves most of the information.

            Usage :
                FCN8 = Semantic_Segmentation.models.FCN.fcn8(input_shape,no_classes,base_model_name,pretrained_weights,regularizer,summary)
                model = FCN8.build()

            # Arguments
                input_shape : size of the input image ,in tuple.
                n_classes : the number of target class.
                            dtype --> int
                base_model : name of the pre-trained cnn model on top of which FCN is built.
                            dtype --> string  default --> 'vgg16'
                weight_path : path to the pre_trained weight files.
                            dtype --> string  default --> 'imagenet'
                regularizer : the regularizing value, it uses L2 regularizers on the kernel/filters.
                            dtype --> float   default -->None
                summary : If True prints the model summary
                                default --> True
        """

        ## NOTE : It wraps the properties of FCN32, same arguments and encoder part but it uses diff decoder function
        ##since the decoder includes 2 skip connections.

        build = lambda self: self.__decoder(super().encoder())

        def __decoder(self,model_en):

            skip_1,skip_2 = super().skip_connections(model_en)

            layer_decoder = Conv2DTranspose(skip_1.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv1')(model_en.output)
            layer_decoder = Add(name='skip1')([layer_decoder, skip_1])
            layer_decoder = Conv2DTranspose(skip_2.get_shape().as_list()[-1], (4, 4), strides=(2, 2), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv2')(layer_decoder)
            layer_decoder = Add(name='skip2')([layer_decoder, skip_2])
            layer_decoder = Conv2DTranspose(self.n_classes, (16, 16), strides=(8, 8), padding='same',
                                            kernel_regularizer=super().weight_decay(self.regularizer), name='deconv3')(layer_decoder)
            output = Activation('softmax',name='softmax')(layer_decoder)

            model = Model(model_en.input,output,name='FCN8-'+self.base_model)
            if self.summary: print(model.summary())
            return model






In [0]:
class focal_loss:

    """ A loss function similar to cross_entropy

        # Usage
            model.compile('sgd',loss=focal_loss.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """
    c_weights = lambda self,x: 1 if x==None else x
    p_weights = lambda self,x: 1 if x==None else x
    clipping = lambda self,x: K.clip(x, K.epsilon(), 1.-K.epsilon())


    def __init__(self,class_weights=None, pixel_weights=None, gamma=2):
        self.class_weights = class_weights
        self.gamma = gamma
        self.pixel_weights = pixel_weights

    def loss(self,y_true,y_pred):

        """ executes the focal loss

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                fl : mean focal loss for the given batch
         """
        y_pred = self.clipping(y_pred)
        fl = -(K.sum((self.c_weights(self.class_weights) * K.pow(1.-y_pred,self.gamma) * (y_true * K.log(y_pred))),axis=-1))
        fl = K.sum((self.p_weights(self.pixel_weights) * fl),axis=(1,2))
        fl = K.mean(fl, axis=0)
        return fl/1000



class cross_entropy(focal_loss):

    """ Categorical cross_entropy
        NOTE : for binary classification it uses softmax instead sigmoid

        # Usage
            model.compile('sgd',loss=cross_entropy.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """

    ## NOTE - this class inherits the properties of focal_loss class
    def loss(self,y_true,y_pred):

        """ executes the categorical cross-entropy

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                ce : mean cross-entropy for the given batch
        """
        y_pred = super().clipping(y_pred)
        ce = -(K.sum((super().c_weights(self.class_weights) * (y_true * K.log(y_pred))),axis=-1))
        ce = K.sum((super().p_weights(self.pixel_weights) * ce),axis=(1,2))
        ce = K.mean(ce,axis=0)
        return ce/1000



class dice_loss(focal_loss):

    """ Its similar to IoU, dice_coeff = (2*A^B)/A U B  dice_loss= 1- dice_coeff
        # Usage
            model.compile('sgd',loss=dice_loss.loss,.......)

        # Arguments
            class_weights : weights for each class to solve the class imbalance problem.
                            dtype --> array   default --> None
            pixel_weights : weights for each pixels in order to segment certain part of the image clearly.
                            dtype --> array   default --> None
    """

    ## NOTE - this class inherits the properties of focal_loss class
    def loss(self,y_true,y_pred):

        """ executes the dice loss

            # Arguments
                y_true : true class values
                y_pred : predicted class values from the model
            # Returns
                dl : dice loss for the given batch
        """
        y_pred = super().clipping(y_pred)
        intersection = K.sum((super().c_weights(self.class_weights) * y_true * y_pred),axis=-1)
        intersection = K.sum((super().p_weights(self.pixel_weights) * intersection),axis=(0,1,2))
        union = K.sum( (super().c_weights(self.class_weights)*((y_true*y_true) + (y_pred*y_pred)) ),axis=-1)
        union = K.sum((super().p_weights(self.pixel_weights) * union),axis=(0,1,2))
        dl = 1. - ((2*intersection)/union)
        return dl



In [0]:
class Metrics:

    __weighted_method = lambda self,x,y,string,w: (K.sum(x)/K.sum(y)) if string=='inter' else (K.sum(w*x)/K.sum(w*y))
    __avg_method = lambda self,x,y,string,w: K.mean(x/y) if string=='intra' else self.__weighted_method(x,y,string,w)
    clipping = lambda self,x: K.clip(x, K.epsilon(), 1.-K.epsilon())


    def __metrics_base(self,y_true,y_pred):

        """ Base for all the metrics defined below """
        y_true,y_pred = K.flatten(tf.math.argmax(y_true,axis=-1)), K.flatten(tf.math.argmax(y_pred,axis=-1))
        con_mat = K.cast(tf.math.confusion_matrix(y_true,y_pred), K.floatx())
        correct = tf.linalg.diag_part(con_mat)
        total = K.sum(con_mat, axis=-1)
        return correct,total,con_mat


    def accuracy(self,y_true,y_pred):

        """ computes the accuracy

            # Arguments
                y_true : target value
                y_pred : predicted class value
            # Returns
                acc : overall accuracy
        """
        correct,total,_ = self.__metrics_base(y_true,y_pred)
        return ( K.sum(correct) / K.sum(total) )


    def IoU(self, y_true, y_pred,average='inter',weights=None):

        """ Intersection over Union , IoU = A^B/(A U B - A^B)
           Computes the percentage overlap with the target image.

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the IoU score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                IoU score
        """
        _, _, con_mat = self.__metrics_base(y_true, y_pred)
        intersection = tf.linalg.diag_part(con_mat)
        ground_truth_set = K.sum(con_mat, axis=1)
        predicted_set = K.sum(con_mat, axis=0)
        union = ground_truth_set + predicted_set - intersection
        return self.__avg_method(intersection,union,average,weights)


    def recall(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the recall score over each given class and gives the overall score.  recall = TP/TP+FN

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the recall score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                recall score
        """
        correct,total,_ = self.__metrics_base(y_true,y_pred)
        return self.__avg_method(correct,total,average,weights)


    def precision(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the precision over each given class and returns the overall score.  precision = TP/TP+FP

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the precision score overall  'intra' --> computes the score for each calss and computes the average
                        'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                precision score
        """
        correct,_,con_mat = self.__metrics_base(y_true,y_pred)
        total = K.sum(con_mat,axis=0)
        return self.__avg_method(correct,total,average,weights)


    def f1score(self,y_true,y_pred,average='inter',weights=None):

        """ Computes the f1 score over each given class and returns the overall score.  f1 = (2*precision*recall)/(precision+recall)

            # Arguments
                y_true : target value
                y_pred : predicted class value
                average : 'inter' --> computes the f1 score overall  'intra' --> computes the score for each calss and computes the average
                            'weighted' --> computes the weighted average , useful for imabalanced class.
                weights :  only if average is specified 'weighted', weights for the respective classes.
            # Returns
                 f1 score
        """
        precision = self.precision(y_true,y_pred,average,weights)
        recall = self.recall(y_true,y_pred,average,weights)
        return ((2*precision*recall)/(precision+recall))



    def dice_coeffiecient(self,y_true,y_pred,average='inter',weights=None):
        """ Computes the dice score over each given class and returns the overall score.

                # Arguments
                    y_true : target value
                    y_pred : predicted class value
                    average : 'inter' --> computes the dice score overall  'intra' --> computes the score for each calss and computes the average
                                    'weighted' --> computes the weighted average , useful for imabalanced class.
                    weights :  only if average is specified 'weighted', weights for the respective classes.
                # Returns
                    dice score
                """

        y_pred = self.clipping(y_pred)
        intersection = 2 * K.sum((y_true * y_pred),axis=(0,1,2))
        union = K.sum( (y_true*y_true) + (y_pred*y_pred),axis=(0,1,2))
        return self.__avg_method(intersection,union,average,weights)

In [0]:
def data_convert(data1,data2,data3,data4):
  #for i in tqdm(range(data1.shape[0])):
  #    for j in range(data1.shape[1]):
  #      for k in range(data1.shape[2]):
  #        for c in range(12):
  #          if max(abs(data1[i,j,k]-data2[c])) == 0: 
  #            data4[i,j,k] = data3[c]
  #            break
  #return data4  
#above changed to below for faster execution
  for i in tqdm(range(n_classes)): 
    data4[(np.where((np.sum(abs(data1-data2[i]),axis=-1)) == 0))] = data3[i]
  return data4
      
def freeze_layers(n_layers):
    for i in range(n_layers):
      model.layers[i].trainable = False

In [0]:
## color map instruction
label_value = np.array([[0,0,0],[70,70,70],[153,153,190],[160, 170, 250],[153, 153, 153],[50, 234, 157],[128, 64, 128],[232, 35, 244],[35, 142, 107],[142, 0, 0],[156, 102, 102],[0, 220, 220]])
label_class = np.array(['Unlabeled','Building','Fence','Other','Pole','Road line','Road','Sidewalk','Vegetation','Car','Wall','Traffic sign'])


In [0]:
#List the files
image_list = os.listdir(path_x)
mask_list = os.listdir(path_y)
temp_image_list=image_list#Creating temp variable so to align the same file names
#add   image path
image_list = [path_x+i for i in image_list]
mask_list = [path_y+i for i in temp_image_list]#using temp_image_list, otherwise file names not same as in RGB and SEG

#Load image and resize
def loadImagesCV2(image_list,xsizepercent,ysizepercent):
  images = []
  for n in  tqdm(range(len(image_list))):
      img = cv2.imread(image_list[n],cv2.IMREAD_COLOR)
      res=cv2.resize(img, (0, 0), fx = xsizepercent, fy =ysizepercent)
      images.append(res)
  return images

## reading the data from image
data_x = np.array(loadImagesCV2(image_list,lngImageSize,lngImageSize))
data_y =np.array(loadImagesCV2(mask_list,lngImageSize,lngImageSize))


100%|██████████| 479/479 [00:03<00:00, 159.30it/s]


In [0]:
data_y.shape

(479, 384, 384, 3)

In [0]:
X=data_x
Y=data_y
data_x,data_y = data_x[0:400],data_y[0:400]
data_y.shape

(400, 384, 384, 3)

In [0]:
## getting the data(target) ready as one-hot encoded before passing to the model
data_y_1hot = np.zeros([data_y.shape[0],data_y.shape[1],data_y.shape[2],n_classes])
label_1hot = tf.one_hot([0,1,2,3,4,5,6,7,8,9,10,11],depth = n_classes)
label_1hot = K.eval(label_1hot)

data_y_1hot = data_convert(data_y,label_value,label_1hot,data_y_1hot)

100%|██████████| 12/12 [00:24<00:00,  2.01s/it]


In [0]:
## after this build a model using the above functions split the data into train and test and run it using different loss functions and metrics which are available above
## data_x --> for input features,  data_y_1hot --> for target

In [0]:
train_x,train_y = data_x[0:lngTrainsize],data_y_1hot[0:lngTrainsize]
val_x,val_y = data_x[lngTrainsize:lngTrainsize+lngValSize],data_y_1hot[lngTrainsize:lngTrainsize+lngValSize]
test_x,test_y = data_x[lngTrainsize+lngValSize:lngTrainsize+lngValSize+lngTestSize],data_y_1hot[lngTrainsize+lngValSize:lngTrainsize+lngValSize+lngTestSize]

In [0]:
train_x,val_x = train_x/255 , val_x/255

In [0]:
#get batchnormalization
if (strBatchNorm=="Yes"):
  batchNorm=True
else:
  batchNorm=False


if strModelName=="Unet":
  fcn = UNET.unet((data_x.shape[1],data_x.shape[2],3),n_classes,0.001,batchNorm)
if (strModelName=="FCN32"):
  fcn = FCN.fcn32((data_x.shape[1],data_x.shape[2],3),n_classes,strBaseModel,"imagenet",0.001)
if (strModelName=="FCN16"):
  fcn = FCN.fcn16((data_x.shape[1],data_x.shape[2],3),n_classes,strBaseModel,"imagenet",0.001)
if (strModelName=="FCN8"):
  fcn = FCN.fcn8((data_x.shape[1],data_x.shape[2],3),n_classes,  strBaseModel,"imagenet",0.001)

In [0]:
(data_x.shape[1],data_x.shape[2],3),n_classes

((384, 384, 3), 12)

In [8]:
model = fcn.build()

Model: "Unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 512, 512, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 512, 512, 64) 36928       conv2d_20[0][0]                  
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 512, 512, 64) 256         conv2d_21[0][0]                  
_______________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="Unet", inputs=Tensor("in..., outputs=Tensor("so...)`


In [0]:
import keras

In [0]:
opt =  keras.optimizers.RMSprop()

In [0]:
freeze_layers(lngFrozenLayers)  ## here 5 is the no of layers needed to be freezed
#get loss function
if (strLossFunction=="focal"):
  fnLoss=focal_loss().loss
if (strLossFunction=="cross entropy"):
  fnLoss=cross_entroy()
if (strLossFunction=="dice"):
  fnLoss=dice_loss().loss
model.compile(opt,loss = fnLoss,metrics=[Metrics().accuracy,Metrics().IoU])

In [0]:
epochs=  lngepochs
batchsize=8

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=1000)
history=model.fit(train_x,train_y,batchsize,epochs, validation_data=(val_x,val_y),callbacks=[es])

Train on 300 samples, validate on 50 samples
Epoch 1/50
300/300 [==============================] - 36s 120ms/step - loss: 1541.0254 - accuracy: 0.3366 - IoU: 0.2030 - val_loss: 1645.4482 - val_accuracy: 0.3169 - val_IoU: 0.1888
Epoch 2/50
300/300 [==============================] - 26s 88ms/step - loss: 758.4328 - accuracy: 0.3468 - IoU: 0.2117 - val_loss: 153.9011 - val_accuracy: 0.5869 - val_IoU: 0.4189
Epoch 3/50
300/300 [==============================] - 26s 88ms/step - loss: 137.1530 - accuracy: 0.5171 - IoU: 0.3599 - val_loss: 121.4088 - val_accuracy: 0.4912 - val_IoU: 0.3272
Epoch 4/50
300/300 [==============================] - 26s 88ms/step - loss: 152.5690 - accuracy: 0.6460 - IoU: 0.4961 - val_loss: 98.4451 - val_accuracy: 0.7223 - val_IoU: 0.5703
Epoch 5/50
300/300 [==============================] - 26s 88ms/step - loss: 102.8369 - accuracy: 0.7260 - IoU: 0.5823 - val_loss: 95.4578 - val_accuracy: 0.7344 - val_IoU: 0.5845
Epoch 6/50
300/300 [==============================] - 

In [0]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.rcParams["figure.figsize"] = (2,2)
plt.plot(history.history['accuracy'])
_=plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(logPath+"myplot1.png")
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(logPath+"myplot2.png")
plt.show()
# summarize history for loss
plt.plot(history.history['IoU'])
plt.plot(history.history['val_IoU'])
plt.title('Model IOU')
plt.ylabel('IoU')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(logPath+"myplot3.png")
plt.show()

In [0]:
pred = model.predict(test_x)

In [0]:
pred[pred > 0.5] = 1
pred[pred<0.5] = 0

In [0]:
predict = np.zeros((test_x.shape[0],data_x.shape[1],data_x.shape[2],3))

In [0]:
predict = data_convert(pred,label_1hot,label_value,predict)

In [0]:
import matplotlib.pyplot as plt

In [0]:
from google.colab.patches import cv2_imshow
#cv2_imshow(data_y[1])

In [0]:
#cv2_imshow(data_y[8])

In [0]:
def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = (np.max([ha, hb]))
    total_width = (wa+wb)
    new_img = np.zeros(shape=(max_height, total_width, 3))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img
 
output = concat_images(data_y[lngTrainsize+lngValSize+16],predict[16])

#plt.imshow(output)
cv2_imshow(output)

#plt.show()

In [0]:
# Convert to Unix timestamp
d1_ts = time.mktime(stTime.timetuple())
d2_ts = time.mktime(datetime.now().timetuple())

# They are now in seconds, subtract and then divide by 60 to get minutes.
minute= int(d2_ts-d1_ts) / 60

In [0]:
trainable_count = count_params(model.trainable_weights)
non_trainable_count = count_params(model.non_trainable_weights)

In [0]:
# Update Excel Report
if (path.exists(logPath+"Logfile.xlsx")==True): #File exist, open it
  wb = openpyxl.load_workbook(logPath+"Logfile.xlsx")
  ws = wb.active
else: #file not available, create it
  wb = Workbook()
  ws = wb.active
  ws['A1']="Run Date"
  ws['B1']="Model"
  ws['C1']="Image Size"
  ws['D1']="Epocs"
  ws['E1']="Train Size"
  ws['F1']="Train Accuracy"
  ws['G1']="Train Loss"
  ws['H1']="Test Size"
  ws['I1']="Test Accuracy"
  ws['J1']="Test Loss"
  ws['K1']="IOU"
  ws['L1']="Time taken (min)"
  ws['M1']="Trainable Params"
  ws['N1']="Non-Trainable Params"
  ws['O1']="Batch size"
  ws['P1']="Optimizer"
  ws['Q1']="Optimizer Params"
  ws['R1']="Loss Function"
  ws['S1']="Comments"
  ws['T1']="EarlyStopping Epoch"
  ws['U1']="BatchNorm"
  ws['V1']="FrozenLayers"
colummn=ws['A']
lastRow=len(colummn)+1

In [0]:
ws['B'+str(lastRow)]=model.name #"Unet" #Model
ws['C'+str(lastRow)]=str(data_x.shape[1])+str('x')+str(data_x.shape[2]) #Image Size
ws['E'+str(lastRow)]=train_x.shape[0] #"Train Size"
ws['H'+str(lastRow)]=val_x.shape[0] #"Test Size" 
ws['D'+str(lastRow)]=epochs #"Epocs"
ws['O'+str(lastRow)]=batchsize #"Batch size"
ws['R'+str(lastRow)]=str(fnLoss) #"focal_loss" #"Loss Function
ws['F'+str(lastRow)]=history.history['accuracy'][-1] #"Train Accuracy"
ws['G'+str(lastRow)]=history.history['loss'][-1] #"Train Loss" 
ws['I'+str(lastRow)]=history.history['val_accuracy'][-1] #"Test Accuracy"
ws['J'+str(lastRow)]=history.history['val_loss'][-1] #"Test Loss"
ws['K'+str(lastRow)]="IOU"
ws['T'+str(lastRow)]=str(len(history.history['accuracy']))
ws['A'+str(lastRow)] = dt_string #Run Date
ws['L'+str(lastRow)] = minute #Time Take

ws['M'+str(lastRow)] = trainable_count #Trainable Params
ws['N'+str(lastRow)] = non_trainable_count #Non-Trainable Params
ws['P'+str(lastRow)] =str(model.optimizer) #Optimizer
ws['Q'+str(lastRow)] =str(model.optimizer.get_config()) #Optimizer Params
ws['S'+str(lastRow)] ="With Convert function as new and with early stopping" #comments

ws['U'+str(lastRow)] = strBatchNorm
ws['V'+str(lastRow)] = lngFrozenLayers

img1 = openpyxl.drawing.image.Image(logPath+"myplot1.png")
img2 = openpyxl.drawing.image.Image(logPath+"myplot2.png")
img3 = openpyxl.drawing.image.Image(logPath+"myplot3.png")
ws.add_image(img1 ,'X'+str(lastRow))
ws.add_image(img2,'AC'+str(lastRow))
ws.add_image(img3,'AH'+str(lastRow))

output=cv2.resize(output, (0, 0), fx = 0.25, fy =0.25)
cv2.imwrite(logPath+'output.png',output)
img1 = openpyxl.drawing.image.Image(logPath+"output.png")
ws.add_image(img1,'AL'+str(lastRow))

ws.row_dimensions[lastRow].height = 120
wb.save(logPath+"Logfile.xlsx")

In [0]:
wbP.save(logPath+"ModelParameters.xlsx")  #run parameter marker, Saving at the end as if program crashes, it can run the same again

In [0]:
exit() #added this so that each time colab keeps some memory, reruning would crash.  using this now memory is reset back